# The Battle of Neighborhoods

## Introduction/Business Problem

Bubble tea is growing more and more popular, bubble tea stores are often have a long queues in some hot business area in Beijing. Asian cuisine is also very popular in the US, especially cities full of Asian immigrants, like the Silicon Valley in California. This project aims to find the best location to open a business in a city in Silicon Valley, San Jose, CA.

It’s important to know if it is profitable prior to launching any business, so this report will try to gather data about similar businesses in the neighborhood and potential customer distribution, competitors, etc.  The data can be used as part of a business plan to support making decision. 

Bubble tea is growing more and more popular, bubble tea stores are often have a long queues in some hot business area in Beijing. Asian cuisine is also very popular in the US, especially cities full of Asian immigrants, like the Silicon Valley in California. This project aims to find the best location to open a business in a city in Silicon Valley, San Jose, CA.

It’s important to know if it is profitable prior to launching any business, so this report will try to gather data about similar businesses in the neighborhood and potential customer distribution, competitors, etc.  The data can be used as part of a business plan to support making decision. 


## Data Description

I will use Foursquare API to find venues:  

For competitor study  

-  Existing Bubble Tea shops around the neighbourhoods in San Jose  
-  Similar businesses around the neighbourhoods in San Jose 

For potential customer distribution study 

-  Universities around the neighbourhoods in San Jose  
-  Companies around the neighbourhoods in San Jose 

Data visualization: 
I will use folium the map rendering library to visualize the venues 


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot
import seaborn as sns
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [6]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [7]:
CLIENT_ID = 'VLEDCUIZD1CXTQVN0GJPI5D4TCWV3PPW32Y52LWOUPS1CBZK' # My Foursquare ID
CLIENT_SECRET = 'HFONPPRLXX35IYQB2ZJJMTTMR1VCSW2DJ4FAM0MFMTQSECWP' # My Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: VLEDCUIZD1CXTQVN0GJPI5D4TCWV3PPW32Y52LWOUPS1CBZK
CLIENT_SECRET:HFONPPRLXX35IYQB2ZJJMTTMR1VCSW2DJ4FAM0MFMTQSECWP


In [8]:
address = 'San Jose, CA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.3361905 -121.8905833


In [9]:
search_query = 'Bubble Tea'
radius = 500
print(search_query + ' .... OK!')

Bubble Tea .... OK!


In [10]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VLEDCUIZD1CXTQVN0GJPI5D4TCWV3PPW32Y52LWOUPS1CBZK&client_secret=HFONPPRLXX35IYQB2ZJJMTTMR1VCSW2DJ4FAM0MFMTQSECWP&ll=37.3361905,-121.8905833&v=20180604&query=Bubble Tea&radius=500&limit=30'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e85f7f9b57e88001b5c7e8c'},
 'response': {'venues': [{'id': '4f445a3619836ed00193a03f',
    'name': "Ola's Exotic Coffee and Tea",
    'location': {'address': '12 S 1st St',
     'lat': 37.335864,
     'lng': -121.890378,
     'labeledLatLngs': [{'label': 'display',
       'lat': 37.335864,
       'lng': -121.890378}],
     'distance': 40,
     'postalCode': '95113',
     'cc': 'US',
     'city': 'San Jose',
     'state': 'CA',
     'country': 'United States',
     'formattedAddress': ['12 S 1st St',
      'San Jose, CA 95113',
      'United States']},
    'categories': [{'id': '4d4b7105d754a06374d81259',
      'name': 'Food',
      'pluralName': 'Food',
      'shortName': 'Food',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1585838004',
    'hasPerk': False},
   {'id': '5a9f59548d0a537c2fc45e0d',
    'name': 'Tea Alley',
    'location': {'add

In [12]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f445a3619836ed00193a03f,12 S 1st St,US,San Jose,United States,NaN,40,"[12 S 1st St, San Jose, CA 95113, United States]","[{'label': 'display', 'lat': 37.335864, 'lng':...",37.335864,-121.890378,NaN,95113,CA,Ola's Exotic Coffee and Tea,v-1585838004,NaN
1,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,5a9f59548d0a537c2fc45e0d,40 S 1st St,US,San Jose,United States,NaN,157,"[40 S 1st St, San Jose, CA 95113, United States]","[{'label': 'display', 'lat': 37.33514500000000...",37.335145,-121.889390,NaN,95113,CA,Tea Alley,v-1585838004,NaN
2,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3276ca19836c91c7da567f,80 S 1st St,US,San Jose,United States,NaN,191,"[80 S 1st St, San Jose, CA 95113, United States]","[{'label': 'display', 'lat': 37.334695, 'lng':...",37.334695,-121.889525,NaN,95113,CA,Pochi Tea Station,v-1585838004,NaN
3,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,5e3b4fb24c4a8500088948d4,150 S 1st St,US,San Jose,United States,NaN,329,"[150 S 1st St, San Jose, CA 95113, United States]","[{'label': 'display', 'lat': 37.333765, 'lng':...",37.333765,-121.888445,NaN,95113,CA,Tea Villa,v-1585838004,NaN
4,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",884781,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/breaktime-c...,False,5aca38b03e67410835ea56e3,110 E San Fernando St,US,San Jose,United States,4th Street,357,"[110 E San Fernando St (4th Street), San Jose,...","[{'label': 'display', 'lat': 37.33553757451000...",37.335538,-121.886625,NaN,95112,CA,Breaktime Cafe & Tea,v-1585838004,NaN


In [24]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Vegan DishCrawl,Vegetarian / Vegan Restaurant,NaN,US,San Jose,United States,NaN,525,"[San Jose, CA 95112, United States]","[{'label': 'display', 'lat': 37.336669, 'lng':...",37.336669,-121.884674,NaN,95112,CA,50a459e9e4b0cbb1fa9a2efb
1,Vegan Donuts & Cafe,Vegetarian / Vegan Restaurant,449 E Santa Clara Street,US,San Jose,United States,10th Street,966,"[449 E Santa Clara Street (10th Street), San J...","[{'label': 'display', 'lat': 37.340736, 'lng':...",37.340736,-121.881280,Horace Mann,95113,CA,5bc3db30da5ede002c80d49b
2,Good Karma,Vegetarian / Vegan Restaurant,37 S 1st St,US,San Jose,United States,NaN,90,"[37 S 1st St, San Jose, CA 95113, United States]","[{'label': 'display', 'lat': 37.33547578302974...",37.335476,-121.890094,NaN,95113,CA,4a55443af964a520f3b31fe3
3,Vegan Veganos,Food Truck,NaN,US,San Jose,United States,NaN,4001,"[San Jose, CA 95070, United States]","[{'label': 'display', 'lat': 37.325695, 'lng':...",37.325695,-121.933821,NaN,95070,CA,5cfc096ef5e9d7002c2142b6
4,Loving Hut,Vegetarian / Vegan Restaurant,2855 Stevens Creek Blvd Ste 2459,US,San Jose,United States,NaN,4887,"[2855 Stevens Creek Blvd Ste 2459, San Jose, C...","[{'label': 'display', 'lat': 37.32614245482791...",37.326142,-121.944330,NaN,95050,CA,5274680711d2508b2aa860a6


In [25]:
dataframe_filtered.name



0        Vegan DishCrawl
1    Vegan Donuts & Cafe
2             Good Karma
3          Vegan Veganos
4             Loving Hut
Name: name, dtype: object

In [27]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around San Jose

# add a red circle marker to represent San Jose
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='San Jose',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Vegan restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [28]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8552a7923935001b0dacba'},
 'response': {'venues': []}}

In [29]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [30]:
trending_venues_df

'No trending venues are available at the moment!'